In [138]:
import os

import pandas as pd
import seaborn as sns
from dotenv import load_dotenv
from matplotlib import pyplot as pl
# from read_gsheet import get_specific_range

In [139]:
# get_specific_range('unpreparing_stady_data')

In [140]:
df=pd.read_csv('unpreparing_stady_data.csv',header=1, date_format=True, dayfirst=True).drop_duplicates()
df

,Вид активности,Спринт,Модули\Спринты,Дата,Утро,Вечер,Доп занятия,Название
0,1.0,Спринт 1,Спринт 1,26.05.2025,125.0,109.0,NaN,NaN
1,1.0,Спринт 1,NaN,27.05.2025,124.0,125.0,NaN,NaN
2,1.0,Спринт 1,NaN,28.05.2025,103.0,NaN,NaN,NaN
3,1.0,Спринт 1,NaN,29.05.2025,NaN,NaN,NaN,NaN
4,1.0,Спринт 1,NaN,30.05.2025,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
396,3.0,Спринт 16,NaN,NaN,NaN,NaN,NaN,NaN
397,2.0,Проект модуля 5,NaN,NaN,NaN,NaN,NaN,NaN
401,2.0,Проект модуля 6,NaN,NaN,NaN,NaN,NaN,NaN
404,2.0,Проект модуля 7,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 390 entries, 0 to 407
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Вид активности  337 non-null    float64
 1   Спринт          342 non-null    object 
 2   Модули\Спринты  29 non-null     object 
 3   Дата            355 non-null    object 
 4   Утро            58 non-null     float64
 5   Вечер           49 non-null     float64
 6   Доп занятия     32 non-null     float64
 7   Название        31 non-null     object 
dtypes: float64(4), object(4)
memory usage: 27.4+ KB


### Приведем все данные по активностям к единому виду

In [142]:
df=df.rename(columns={'Вид активности':'rank',
                   'Спринт':'activity_subtype_name',
                    'Дата':'activity_date',
                    'Утро':'morning',
                    'Вечер':'evening',
                    'Доп занятия':'add_study_duration',
                    'Название':'add_study_type'}).drop(columns=['Модули\Спринты','rank'])

In [143]:
df.activity_date = pd.to_datetime(df['activity_date'], format='%d.%m.%Y')
df.activity_date = df.activity_date.dt.date

In [144]:
df=df.fillna(0)

df

,activity_subtype_name,activity_date,morning,evening,add_study_duration,add_study_type
0,Спринт 1,2025-05-26,125.0,109.0,0.0,0
1,Спринт 1,2025-05-27,124.0,125.0,0.0,0
2,Спринт 1,2025-05-28,103.0,0.0,0.0,0
3,Спринт 1,2025-05-29,0.0,0.0,0.0,0
4,Спринт 1,2025-05-30,0.0,0.0,0.0,0
...,...,...,...,...,...,...
396,Спринт 16,0,0.0,0.0,0.0,0
397,Проект модуля 5,0,0.0,0.0,0.0,0
401,Проект модуля 6,0,0.0,0.0,0.0,0
404,Проект модуля 7,0,0.0,0.0,0.0,0


### Перевод данных из полей morning, evening и add_study в отдельные строки. Приведение к формату duration - daypart  

In [145]:
morning_df=df.query('morning != 0 & activity_date !=0')\
    .drop(columns=['add_study_duration','add_study_type','evening'])
morning_df=morning_df.replace('Итоговый проект 1','Проект модуля 1')
morning_df['daypart']='Утро'
morning_df['morning']=morning_df['morning'].astype(int)
morning_df=morning_df.rename(columns={'morning':'duration'})
morning_df.head()

,activity_subtype_name,activity_date,duration,daypart
0,Спринт 1,2025-05-26,125,Утро
1,Спринт 1,2025-05-27,124,Утро
2,Спринт 1,2025-05-28,103,Утро
15,Спринт 2,2025-06-10,112,Утро
16,Спринт 2,2025-06-11,129,Утро


In [146]:
evening_df=df.query('evening != 0 & activity_date !=0')\
    .drop(columns=['add_study_duration','add_study_type','morning'])
evening_df=evening_df.replace('Итоговый проект 1','Проект модуля 1')
evening_df['daypart']='Вечер'
evening_df['evening']=evening_df['evening'].astype(int)
evening_df=evening_df.rename(columns={'evening':'duration'})
evening_df.head()

,activity_subtype_name,activity_date,duration,daypart
0,Спринт 1,2025-05-26,109,Вечер
1,Спринт 1,2025-05-27,125,Вечер
14,Спринт 2,2025-06-09,62,Вечер
16,Спринт 2,2025-06-11,95,Вечер
17,Спринт 2,2025-06-12,123,Вечер


In [147]:
web_df=df.query('add_study_type == "Вебинар"')
web_df=web_df.drop(columns='activity_subtype_name')\
     .rename(columns={'add_study_type':'activity_subtype_name'})
web_df['duration']=web_df['add_study_duration']
web_df['daypart']='Вечер'
web_df=web_df.drop(columns=['morning','evening','add_study_duration'])
web_df['duration']=web_df['duration'].astype(int)
web_df

,activity_date,activity_subtype_name,duration,daypart
10,2025-06-05,Вебинар,60,Вечер
36,2025-07-01,Вебинар,75,Вечер
37,2025-07-02,Вебинар,117,Вечер
51,2025-07-16,Вебинар,100,Вечер
65,2025-07-30,Вебинар,110,Вечер
91,2025-08-25,Вебинар,120,Вечер
99,2025-09-02,Вебинар,90,Вечер
106,2025-09-09,Вебинар,120,Вечер


In [148]:
add_study_df=df.query('add_study_type != 0 & activity_date !=0 & add_study_type != "Вебинар"')
add_study_df=add_study_df.drop(columns='activity_subtype_name')\
    .rename(columns={'add_study_type':'activity_subtype_name','duration':'add_study_duration'})
# add_study_df['duration']=add_study_df['duration'].astype(int)
add_study_df['duration_evening']=add_study_df.add_study_duration
add_study_df['duration_morning']=0
add_study_df.loc[38,'duration_evening']=120
add_study_df.loc[39,'duration_evening']=20
add_study_df.loc[52,'duration_evening']=30
add_study_df.loc[52,'duration_morning']=120
add_study_df.loc[63,'duration_evening']=60
add_study_df.loc[63,'duration_morning']=120
add_study_df.loc[64,'duration_morning']=120
add_study_df.loc[64,'duration_evening']=120
add_study_df.loc[66,'duration_morning']=120
add_study_df.loc[66,'duration_evening']=60
add_study_df.loc[67,'duration_morning']=120
add_study_df.loc[67,'duration_evening']=120
add_study_df.loc[74,'duration_morning']=120
add_study_df.loc[74,'duration_evening']=120
add_study_df.loc[76,'duration_morning']=120
add_study_df.loc[76,'duration_evening']=60
add_study_df.loc[77,'duration_morning']=120
add_study_df.loc[77,'duration_evening']=120
add_study_df.loc[78,'duration_morning']=120
add_study_df.loc[78,'duration_evening']=240
add_study_df.loc[79,'duration_morning']=120
add_study_df.loc[79,'duration_evening']=120
add_study_df.loc[86,'duration_morning']=60
add_study_df.loc[86,'duration_evening']=60

add_study_df.head()


,activity_date,morning,evening,add_study_duration,activity_subtype_name,duration_evening,duration_morning
38,2025-07-03,0.0,0.0,120.0,sql-ex.ru (задачи),120.0,0
39,2025-07-04,0.0,0.0,20.0,sql-ex.ru (задачи),20.0,0
41,2025-07-06,0.0,0.0,46.0,sql-ex.ru (задачи),46.0,0
50,2025-07-15,0.0,0.0,55.0,sql-ex.ru (задачи),55.0,0
52,2025-07-17,0.0,0.0,150.0,lab.karpov.courses,30.0,120


In [149]:
add_study_morning=add_study_df.query('duration_morning != 0')\
    .drop(columns=['morning','evening','add_study_duration','duration_evening'])\
    .rename(columns={'duration_morning':'duration'})
add_study_morning.duration=pd.to_numeric(add_study_morning.duration)
add_study_morning['daypart']='Утро'
add_study_morning.head()

,activity_date,activity_subtype_name,duration,daypart
52,2025-07-17,lab.karpov.courses,120,Утро
63,2025-07-28,lab.karpov.courses,120,Утро
64,2025-07-29,lab.karpov.courses,120,Утро
66,2025-07-31,lab.karpov.courses,120,Утро
67,2025-08-01,lab.karpov.courses,120,Утро


In [150]:
add_study_evening=add_study_df.query('duration_evening != 0')\
    .drop(columns=['morning','evening','add_study_duration','duration_morning'])\
    .rename(columns={'duration_evening':'duration'})
add_study_evening['duration']=add_study_evening['duration'].astype(int)
add_study_evening['daypart']='Вечер'
add_study_evening.head()

,activity_date,activity_subtype_name,duration,daypart
38,2025-07-03,sql-ex.ru (задачи),120,Вечер
39,2025-07-04,sql-ex.ru (задачи),20,Вечер
41,2025-07-06,sql-ex.ru (задачи),46,Вечер
50,2025-07-15,sql-ex.ru (задачи),55,Вечер
52,2025-07-17,lab.karpov.courses,30,Вечер


### Объединяем все очищенные таблицы в одну и сохраняем в .csv

In [151]:
clear_df=pd.concat([morning_df,evening_df,add_study_evening,add_study_morning,web_df])
clear_df.head()


,activity_subtype_name,activity_date,duration,daypart
0,Спринт 1,2025-05-26,125,Утро
1,Спринт 1,2025-05-27,124,Утро
2,Спринт 1,2025-05-28,103,Утро
15,Спринт 2,2025-06-10,112,Утро
16,Спринт 2,2025-06-11,129,Утро


### Подготовим таблицу 'activity_type' для загрузки в базу данных

In [152]:
activity_types = pd.DataFrame({
    'activity_type_name': ['Основное обучение','Проект','Специализация PA','Специализация BI','Дипломная работа','Дополнительные занятия','Сторонние курсы','Pet-проекты','Учебные кейсы']
    })
activity_types

,activity_type_name
0,Основное обучение
1,Проект
2,Специализация PA
3,Специализация BI
4,Дипломная работа
5,Дополнительные занятия
6,Сторонние курсы
7,Pet-проекты
8,Учебные кейсы


### Подготовим таблицу 'activity_subtype' для загрузки в базу данных

In [175]:
activity_subtypes = pd.DataFrame({
    'activity_subtype_name': ['Спринт 1','Спринт 2','Спринт 3','Спринт 4','Спринт 5','Спринт 6','Спринт 7','Спринт 8','Спринт 9','Спринт 10','Спринт 11','Проект модуля 1','Проект модуля 2','Проект модуля 3','Выпускной проект',
                              'Спринт 12','Спринт 13','Спринт 14','Спринт 15','Спринт 16','Спринт 12','Спринт 13','Спринт 14','Спринт 15','Спринт 16','Проект модуля 4','Проект модуля 5','Проект модуля 6','Проект модуля 7',
                              'Карьерный трек','BigData','ClickHouse','PySpark','AirFlow','Вебинар',
                              'AD Machine Learning','AD Линейная алгебра','AD Алгоритмы и структуры данных','AD YandexGPT','AD Python Lab','SQL Lab','AD Конструктивное общение','AD Аргументация в рабочей коммуникации','AD Анализ и принятие управленческих решений',
                              'sql-ex.ru (задачи)','lab.karpov.courses','codewars.com','stepik.org','Study scheduler bot',
                              'Кейс Спринта 1','Кейс Спринта 2','Кейс Спринта 3','Кейс Спринта 5','Кейс Спринта 7','Кейс Спринта 8','Кейс Спринта 9','Кейс Спринта 10','Кейс Спринта 11','Курс по DataLens'],
    'norm_time':[1200,1200,1200,1200,1200,1200,1200,1800,1200,1800,1200,1200,660,720,1200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    'activity_type_id':[1,1,1,1,1,1,1,1,1,1,1,2,2,2,5,3,3,3,3,3,4,4,4,4,4,2,2,2,2,1,1,1,1,1,1,6,6,6,6,6,6,6,6,6,7,7,7,7,8,9,9,9,9,9,9,9,9,9,7]
})
activity_subtypes.head()

,activity_subtype_name,norm_time,activity_type_id
0,Спринт 1,1200,1
1,Спринт 2,1200,1
2,Спринт 3,1200,1
3,Спринт 4,1200,1
4,Спринт 5,1200,1


In [169]:
subtype_list = df['activity_subtype_name'].to_frame()\
    .query('activity_subtype_name !=0 & activity_subtype_name != "Внекурсовые" & activity_subtype_name != "Каникулы"')
subtype_list = subtype_list.drop_duplicates().sort_values('activity_subtype_name', ascending=True, ignore_index=True)
subtype_list['norm_time'] = 0
subtype_list = pd.merge(subtype_list,activity_subtypes, how='left', on='activity_subtype_name')\
    .fillna(0)\
    .drop(columns='norm_time_x')\
    .rename(columns={'norm_time_y':'norm_time'})
subtype_list['norm_time'] = subtype_list['norm_time'].astype(int)
subtype_list.head()

,activity_subtype_name,norm_time,activity_type_id
0,AD Machine Learning,0,6.0
1,AD Python Lab,0,6.0
2,AD YandexGPT,0,6.0
3,AD Алгоритмы и структуры данных,0,6.0
4,AD Анализ и принятие управленческих решений,0,6.0


### Подготовим таблицу 'activities' для загрузки в базу данных

In [170]:
activity_subtype_id=activity_subtypes.reset_index()
activity_subtype_id['activity_subtype_id']=activity_subtype_id.index+1
activity_subtype_id=activity_subtype_id.drop(columns=['index'])
activity_subtype_id.head()

,activity_subtype_name,norm_time,activity_type_id,activity_subtype_id
0,Спринт 1,1200,1,1
1,Спринт 2,1200,1,2
2,Спринт 3,1200,1,3
3,Спринт 4,1200,1,4
4,Спринт 5,1200,1,5


In [172]:
load_dotenv()
telegram_id=os.getenv('TG_ID')

activities=pd.merge(clear_df,activity_subtype_id, how='left', on='activity_subtype_name')\
    .drop(columns=['activity_subtype_name','norm_time','activity_type_id'])
activities['telegram_id']=telegram_id
activities


,activity_date,duration,daypart,activity_subtype_id,telegram_id
0,2025-05-26,125,Утро,1,420221927
1,2025-05-27,124,Утро,1,420221927
2,2025-05-28,103,Утро,1,420221927
3,2025-06-10,112,Утро,2,420221927
4,2025-06-11,129,Утро,2,420221927
...,...,...,...,...,...
142,2025-07-16,100,Вечер,35,420221927
143,2025-07-30,110,Вечер,35,420221927
144,2025-08-25,120,Вечер,35,420221927
145,2025-09-02,90,Вечер,35,420221927


### Сохраним все подготовленные таблицы в .csv файлы

In [157]:
activities.to_csv('activities.csv', index=False)
activity_types.to_csv('activity_types.csv', index=False)
activity_subtypes.to_csv('activity_subtypes.csv', index=False)

In [178]:
check_df = pd.read_csv('activities.csv')
check_df = check_df.sort_values('activity_date',ascending=False)
check_df.head(1)

,activity_date,duration,daypart,telegram_id
104,2025-09-13,120,Вечер,420221927


In [160]:
ac_ty_df = pd.read_csv('activity_types.csv')
ac_ty_df

,activity_type_name
0,Основное обучение
1,Проект
2,Специализация PA
3,Специализация BI
4,Дипломная работа
5,Дополнительные занятия
6,Сторонние курсы
7,Pet-проекты
8,Учебные кейсы


In [194]:
ac_sbty_df = pd.read_csv('activity_subtypes.csv').reset_index()
ac_sbty_df['activity_subtype_id'] = ac_sbty_df.index+1
ac_sbty_df 

,index,activity_subtype_name,norm_time,activity_type_id,activity_subtype_id
0,0,Спринт 1,1200,1,1
1,1,Спринт 2,1200,1,2
2,2,Спринт 3,1200,1,3
3,3,Спринт 4,1200,1,4
4,4,Спринт 5,1200,1,5
5,5,Спринт 6,1200,1,6
6,6,Спринт 7,1200,1,7
7,7,Спринт 8,1800,1,8
8,8,Спринт 9,1200,1,9
9,9,Спринт 10,1800,1,10


In [197]:
load_dotenv()
telegram_id=os.getenv('TG_ID')

activities=pd.merge(clear_df,ac_sbty_df, how='left', on='activity_subtype_name')\
    .drop(columns=['activity_subtype_name','norm_time','activity_type_id','index','norm_time'])
activities['telegram_id']=telegram_id
activities.sort_values('activity_date', ascending=False).head(1)


,activity_date,duration,daypart,activity_subtype_id,telegram_id
104,2025-09-13,120,Вечер,49,420221927
